# Installing Dependencies

In [24]:
using DelimitedFiles
using NearestNeighbors: knn, KDTree, BruteTree, BallTree
using LinearAlgebra: norm
using CUDA

# Initial Conditions

In [25]:
R_Agg = 16
t, t_knn = 0, 10

@time X = readdlm("../data/Init/Two_Sphere/$R_Agg.csv", ',', Float64, header=true)[1][:, 1:3];

  0.024195 seconds (565.10 k allocations: 15.185 MiB, 32.50% gc time)


# kNN

In [58]:
println("Generating Trees")
@time brutetree = BruteTree(X')
@time kdtree = KDTree(X')
@time balltree = BallTree(X')

i=1
println("Generating One index array")
@time idxs, _ = knn(brutetree, X[i,:], 14, true)
@time idxs, _ = knn(kdtree, X[i,:], 14, true)
@time idxs, _ = knn(balltree, X[i,:], 14, true);

Generating Trees
  0.000092 seconds (7 allocations: 142.281 KiB)
  0.000924 seconds (29 allocations: 399.188 KiB)
  0.001466 seconds (28 allocations: 418.047 KiB)
Generating One index array
  0.000091 seconds (7 allocations: 544 bytes)
  0.000020 seconds (7 allocations: 544 bytes)
  0.000035 seconds (7 allocations: 544 bytes)


In [60]:
@time function NearNeighbor_Brute(X, t, t_knn)
    # Using kNN for Nearest Neighbors
    if t%t_knn | t == 0
        global brutetree = BruteTree(X')
    end 

    # Loop over all cells to compute displacements
    index, _ = knn(brutetree, X[1,:], 14, true)
    for i in 2:size(X)[1]
        # Scan neighbours
        global idxs, _ = knn(brutetree, X[i,:], 14, true)
        index = hcat(index, idxs)

    end
    return index
end

@time function NearNeighbor_KD(X, t, t_knn)
    # Using kNN for Nearest Neighbors
    if t%t_knn | t == 0
        global kdtree = KDTree(X')
    end 

    # Loop over all cells to compute displacements
    index, _ = knn(kdtree, X[1,:], 14, true)
    for i in 2:size(X)[1]
        # Scan neighbours
        global idxs, _ = knn(kdtree, X[i,:], 14, true)
        index = hcat(index, idxs)

    end
    return index
end

@time function NearNeighbor_Ball(X, t, t_knn)
    # Using kNN for Nearest Neighbors
    if t%t_knn | t == 0
        global balltree = BallTree(X')
    end 

    # Loop over all cells to compute displacements
    index, _ = knn(balltree, X[1,:], 14, true)
    for i in 2:size(X)[1]
        # Scan neighbours
        global idxs, _ = knn(balltree, X[i,:], 14, true)
        index = hcat(index, idxs)

    end
    return index
end

  0.000075 seconds (29 allocations: 2.390 KiB)
  0.000029 seconds (29 allocations: 2.401 KiB)
  0.000021 seconds (29 allocations: 2.403 KiB)


NearNeighbor_Ball (generic function with 1 method)

In [76]:
@time idx = NearNeighbor_Brute(X, t, t_knn)

  0.706148 seconds (36.24 k allocations: 1.921 GiB, 2.62% gc time)


14×6064 Matrix{Int64}:
   1    2    3    4    5    6    7  …  6059  6060  6061  6062  6063  6064
  25   26   27   41   42   42   44     6024  6024  6025  6038  6039  6040
  32   33   35   40   41   43   43     6023  6025  6026  6030  6031  6033
  33   34   34    9   10   10   11     6055  6056  6057  6031  6032  6032
   2    3    2    8    9   11    6     6056  6057  6060  6063  6064  6063
   5    1    7    5    4    7   35  …  6060  6059  6033  6059  6062  6061
   4    5    6   32    6    5    3     6058  6061  6064  6058  6060  6060
  26    6   28    1   33   34   51     6031  6032  6017  6037  6059  6039
  24   27   26   48    2    3   36     6062  6063  6032  6039  6040  6041
  41   25   43   33    1    2   34     6063  6064  6034  6023  6038  6025
 102   42  104   31   49   50  114  …  6015  6016  5954  5962  6024  5964
  19  103   21  111   34   33   50     6032  6031  6018  6045  5963  6047
  20   20   22   49   32   35   52     6030  6033  6016  6044  6046  6046
  40   21   42 

In [78]:
@time idx = NearNeighbor_KD(X, t, t_knn)

  0.226111 seconds (36.27 k allocations: 1.922 GiB, 9.37% gc time)


14×6064 Matrix{Int64}:
   1    2    3    4    5    6    7  …  6059  6060  6061  6062  6063  6064
  25   26   27   40   42   43   44     6024  6025  6026  6038  6039  6040
  33   33   35   41   41   42   43     6023  6024  6025  6030  6031  6033
  32   34   34    9    9   10   11     6055  6057  6057  6031  6032  6032
   2    3    2    8   10   11    6     6056  6056  6060  6063  6064  6063
   4    1    6    5    4    5   35  …  6058  6061  6033  6059  6062  6061
   5    6    7   32    6    7    3     6060  6059  6064  6058  6060  6060
  24    5   26    1   33   34   51     6031  6032  6017  6037  6059  6039
  26   25   28   48    2    3   34     6062  6063  6034  6039  6038  6041
  41   27   43   33    1    2   36     6063  6064  6032  6023  6040  6025
 102   42  104   31   49   50  114  …  6015  6016  5954  5962  6024  5964
  20  103   22  111   34   35   52     6030  6033  6018  6045  5963  6047
  19   20   21   47   32   33   50     6032  6031  6016  6044  6045  6046
  42   21   44 

In [80]:
@time idx = NearNeighbor_Ball(X, t, t_knn)

  0.247762 seconds (36.27 k allocations: 1.922 GiB, 7.34% gc time)


14×6064 Matrix{Int64}:
   1    2    3    4    5    6    7  …  6059  6060  6061  6062  6063  6064
  25   26   27   41   42   43   44     6024  6025  6026  6038  6039  6040
  33   33   35   40   41   42   43     6023  6024  6025  6031  6032  6033
  32   34   34    8    9   11   11     6056  6056  6057  6030  6031  6032
   2    3    2    9   10   10    6     6055  6057  6060  6063  6062  6063
   5    1    6    5    4    7   35  …  6058  6059  6033  6058  6064  6061
   4    5    7   32    6    5    3     6060  6061  6064  6059  6060  6060
  26    6   28    1   33   34   51     6031  6032  6017  6039  6059  6039
  24   27   26   48    2    3   36     6062  6063  6034  6037  6038  6041
  41   25   43   31    1    2   34     6063  6064  6032  6023  6040  6025
 102   42  104   33   49   50  114  …  6015  6016  5954  5962  6024  5964
  20  103   22  111   34   35   52     6030  6033  6018  6045  5963  6047
  19   21   21   49   32   33   50     6032  6031  6016  6044  6045  6046
  42   20   44 